In [1]:
import pickle

In [5]:
with open('./dataset/GPS/patch/0_10_gps.pkl', 'rb') as f:
    data = pickle.load(f, encoding="bytes")

In [6]:
data

,lat,lon,dir,speed,timeinterval
26048,982,18,0,0,182
34446,828,627,54,1131,187
48090,1011,184,165,0,184
70358,595,996,237,51,135
78245,1004,39,0,0,311
...,...,...,...,...,...
49985516,1013,221,260,0,252
50012249,634,858,0,0,180
50018969,1006,154,0,0,180
50040660,1004,33,14640,0,412


In [7]:
with open('./dataset/GPS/beijing_gps_dir_speed_interval_sorted.pkl', 'rb') as f:
    data2 = pickle.load(f, encoding="bytes")

In [8]:
data2

,ID,time,lat,lon,dir,speed,timeinterval
0,0,1228061046,39.71743,116.61815,0,0,NaN
1,0,1228088457,39.71742,116.61798,0,0,177.5
2,0,1228088520,39.71670,116.61420,159,0,150.5
3,0,1228088758,39.71742,116.61798,0,0,272.5
4,0,1228090926,39.71670,116.61428,0,0,354.5
...,...,...,...,...,...,...,...
50041852,28036,1228742522,39.95209,116.30327,23470,0,427.5
50041853,28036,1228742828,39.95206,116.30322,22420,0,429.0
50041854,28036,1228743380,39.95198,116.30315,1820,0,NaN
50041855,28037,1228089568,40.08177,116.23259,16130,1222,NaN


In [8]:
import numpy as np
import pandas as pd
help(np.ndim)
# help(pd.ndim)

Help on function ndim in module numpy:

ndim(a)
    Return the number of dimensions of an array.
    
    Parameters
    ----------
    a : array_like
        Input array.  If it is not already an ndarray, a conversion is
        attempted.
    
    Returns
    -------
    number_of_dimensions : int
        The number of dimensions in `a`.  Scalars are zero-dimensional.
    
    See Also
    --------
    ndarray.ndim : equivalent method
    shape : dimensions of array
    ndarray.shape : dimensions of array
    
    Examples
    --------
    >>> np.ndim([[1,2,3],[4,5,6]])
    2
    >>> np.ndim(np.array([[1,2,3],[4,5,6]]))
    2
    >>> np.ndim(1)
    0

